# `ProteinWorkshop` Tutorial, Part 1 - Training a New Model
![Models](../docs/source/_static/box_models.png)

## Train a new model using the `ProteinWorkshop`

In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

Welcome to the tutorial series for the `ProteinWorkshop`! 

In the `ProteinWorkshop`, we implement numerous [featurisation](https://www.proteins.sh/configs/features) schemes, [datasets](https://www.proteins.sh/configs/dataset) for [self-supervised pre-training](https://proteins.sh/quickstart_component/pretrain.html) and [downstream evaluation](https://proteins.sh/quickstart_component/downstream.html), [pre-training](https://proteins.sh/configs/task) tasks, and [auxiliary tasks](https://proteins.sh/configs/task.html#auxiliary-tasks).

[Processed datasets](https://drive.google.com/drive/folders/18i8rLST6ZICTBu6Q67ClT0KqN9AHeqoW?usp=sharing) and [pre-trained weights](https://drive.google.com/drive/folders/1zK1r8FpmGaqV_QwUJuvDacwSL0RW-Vw9?usp=sharing) are made available. Downloading datasets is not required; upon first run all datasets will be downloaded and processed from their respective source.

The `ProteinWorkshop` encompasses several models as well as pre-trained weights for them so that you can readily use them.

In this tutorial, we show you how you can use what is already available in the protein workshop to train and use models for specific tasks. The `ProteinWorkshop` is structured as a very modular package; we will therefore talk about how to change the different parts of it, like the model, training task, dataset, featurization scheme, etc. in this tutorial. 

Besides using all the different options we provide, you can make use of the modular nature of the `ProteinWorkshop` to add your own models, datasets, featurization schemes, and training tasks. We will show you how to do this in the next tutorials.

To train a new model, you can follow the following 3-step procedure:

1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks
2. Validate the designed training config
3. Use the designed config to train a new model

### 1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks

You can switch out any of these for another available option by replacing the corresponding argument's value in `overrides`:

`cfg = hydra.compose("template", overrides=["encoder=schnet", "task=inverse_folding", "dataset=afdb_swissprot_v4", "features=ca_base", "+aux_task=none"], return_hydra_config=True)`

In [2]:
import os

# Set the environment variable to make only GPU 0 visible
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Now import the necessary librarie
import torch

# Verify GPUs visible to PyTorch
torch_visible_gpus = torch.cuda.device_count()
print("Visible GPUs for PyTorch:", torch_visible_gpus)

Visible GPUs for PyTorch: 1


In [3]:
# Misc. tools
import os

# Hydra tools
import hydra

from hydra.compose import GlobalHydra
from hydra.core.hydra_config import HydraConfig

from proteinworkshop.constants import HYDRA_CONFIG_PATH
from proteinworkshop.utils.notebook import init_hydra_singleton

version_base = "1.2"  # Note: Need to update whenever Hydra is upgraded
init_hydra_singleton(reload=True, version_base=version_base)

path = HYDRA_CONFIG_PATH
rel_path = os.path.relpath(path, start=".")
# print(rel_path)
GlobalHydra.instance().clear()
hydra.initialize(rel_path, version_base=version_base)

cfg = hydra.compose(
    config_name="train",
    overrides=[
        "encoder=schnet",
        "encoder.hidden_channels=512", # Number of channels in the hidden layers
        "encoder.out_dim=32", # Output dimension of the model
        "encoder.num_layers=6", # Number of filters used in convolutional layers
        "encoder.num_filters=128", # Number of convolutional layers in the model
        "encoder.num_gaussians=50", # Number of Gaussian functions used for radial filters
        "encoder.cutoff=10.0", # Cutoff distance for interactions
        "encoder.max_num_neighbors=32", # Maximum number of neighboring atoms to consider
        "encoder.readout=add", # Global pooling method to be used
        "encoder.dipole=False",
        "encoder.mean=null",
        "encoder.std=null",
        "encoder.atomref=null",
        "encoder.pretraining=False",

        # "decoder.graph_label.dummy=True",

        "task=multiclass_graph_classification",
        "dataset=fold_family",
        "dataset.datamodule.batch_size=32",
        "features=ca_angles", 
        "+aux_task=none",
        
        "trainer.max_epochs=150",
        "optimiser=adam",
        "optimiser.optimizer.lr=3e-4",
        "callbacks.early_stopping.patience=10",
        "test=True",
        "scheduler=plateau", # 5 epochs - 0.6 default
        # "+ckpt_path=/home/zhang/Projects/3d/ProteinWorkshop/notebooks/outputs/checkpoints/last.ckpt", # continue training
        ## for test ONLY
        # "task_name=test",  # here
        # "ckpt_path_test=/home/zhang/Projects/3d/proteinworkshop_checkpoints/outputs_pronet_fold_400epochs/checkpoints/epoch_273.ckpt", # here
        # "optimizer.weight_decay=0.5"
        "seed=52",
    ],
    return_hydra_config=True,
)

# Note: Customize as needed e.g., when running a sweep
cfg.hydra.job.num = 0
cfg.hydra.job.id = 0
cfg.hydra.hydra_help.hydra_help = False
cfg.hydra.runtime.output_dir = "outputs"
HydraConfig.instance().set_config(cfg)

[07/31/24 15:53:48] DEBUG    No `.env` file found in project root. Checking for env vars...         ]8;id=536936;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=951452;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#22\22]8;;\

                    DEBUG    No env var `DATA_PATH` found. Setting default...                       ]8;id=227671;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=681141;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#28\28]8;;\

                    INFO     DATA_PATH:                                                             ]8;id=35144;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=814631;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#39\39]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/data                          

[07/31/24 15:53:49] INFO     Hydra initialised at                                                    ]8;id=189925;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py\notebook.py]8;;\:]8;id=133985;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py#53\53]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/config.                       

### 2. Validate the designed training config

This is not strictly necessary, but it is a good idea to validate the config before training. This will check that all the arguments you have provided are valid and that the config is complete.

In [4]:
from proteinworkshop.configs import config

cfg = config.validate_config(cfg)

                    DEBUG    CUDA available: True                                                     ]8;id=635694;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=837827;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#249\249]8;;\

                    DEBUG    Requested GPUs: 1.                                                       ]8;id=454406;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=668321;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#250\250]8;;\

                    DEBUG    GPU count set to: 1                                                      ]8;id=4236;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=758597;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#255\255]8;;\

                    WARNING  You are not using early stopping.                                        ]8;id=393603;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=659579;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#165\165]8;;\

In [5]:
print(cfg.keys())
for key in cfg.keys():
    print(key)
    print(cfg[key])

dict_keys(['hydra', 'env', 'dataset', 'features', 'encoder', 'decoder', 'transforms', 'callbacks', 'optimiser', 'scheduler', 'trainer', 'extras', 'metrics', 'task', 'logger', 'hparams', 'name', 'seed', 'num_workers', 'task_name', 'ckpt_path_test', 'test', 'aux_task'])
hydra
{'run': {'dir': '${env.paths.log_dir}/${task_name}/runs/${now:%Y-%m-%d}_${now:%H-%M-%S}'}, 'sweep': {'dir': '${env.paths.log_dir}/${task_name}/multiruns/${now:%Y-%m-%d}_${now:%H-%M-%S}', 'subdir': '${hydra.job.num}'}, 'launcher': {'_target_': 'hydra._internal.core_plugins.basic_launcher.BasicLauncher'}, 'sweeper': {'_target_': 'hydra._internal.core_plugins.basic_sweeper.BasicSweeper', 'max_batch_size': None, 'params': None}, 'help': {'app_name': '${hydra.job.name}', 'header': '${hydra.help.app_name} is powered by Hydra.\n', 'footer': 'Powered by Hydra (https://hydra.cc)\nUse --hydra-help to view Hydra specific help\n', 'template': '${hydra.help.header}\n== Configuration groups ==\nCompose your configuration from tho

### 3. Use the designed config to train a new model

Now with the config you have designed, you can train a new model. You can also use the `ProteinWorkshop` to evaluate the model on a downstream task.

In [6]:
# import torch
# import torch.nn as nn
# ckpt_path = '/home/zhang/Projects/3d/proteinworkshop_checkpoints/outputs_pronet_pretraining_best@2/checkpoints/epoch_002.ckpt'
# # Assuming `model` is your model and `encoder_weights` is the state_dict of pretrained weights
# print(torch.load(ckpt_path).keys())
# # Load the pretrained state_dict
# checkpoint = torch.load(ckpt_path)
# pretrained_dict = torch.load(ckpt_path)["state_dict"]

# # Create a new state_dict that excludes the final layer parameters
# filtered_dict = {k: v for k, v in pretrained_dict.items() if not k.startswith('encoder.lin_out.')}
# filtered_dict = {k: v for k, v in filtered_dict.items() if not k.startswith('encoder.lins_out.')}
# print(filtered_dict.keys())
# new_checkpoint = {
#     "epoch": checkpoint["epoch"],
#     "global_step": checkpoint["global_step"],
#     "pytorch-lightning_version": checkpoint["pytorch-lightning_version"],
#     "state_dict": filtered_dict,
#     "loops": checkpoint["loops"],
#     "callbacks": checkpoint["callbacks"],
#     "optimizer_states": checkpoint["optimizer_states"],
#     "lr_schedulers": checkpoint["lr_schedulers"],
#     "hparams_name": checkpoint["hparams_name"],
#     "hyper_parameters": checkpoint["hyper_parameters"]
# }

# torch.save(new_checkpoint, '/home/zhang/Projects/3d/proteinworkshop_checkpoints/outputs_pronet_pretraining_best@2/checkpoints/epoch_002_filtered.ckpt')

In [7]:
cfg.get("ckpt_path")

In [9]:
from loguru import logger as log
import torch
import torch.nn as nn
import collections
from proteinworkshop.models.base import BenchMarkModel
model = BenchMarkModel(cfg)

[07/31/24 15:55:03] INFO     Instantiating encoder...                                                   ]8;id=641731;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=460728;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#413\413]8;;\

/home/zhang/miniconda3/envs/3d/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning:

Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.



                    INFO     SchNetModel(hidden_channels=512, num_filters=128, num_interactions=6,      ]8;id=882397;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=750418;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#415\415]8;;\
                             num_gaussians=50, cutoff=10.0)                                                        

                    INFO     Instantiating decoders...                                                  ]8;id=873073;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=423284;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#417\417]8;;\

                    INFO     Building graph_label decoder. Output dim 1195                              ]8;id=640033;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=94292;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#252\252]8;;\

...

                    INFO     ['accuracy', 'f1_score', 'f1_max']                                         ]8;id=811999;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=873194;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#434\434]8;;\

                    INFO     Instantiating featuriser...                                                ]8;id=242395;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=652361;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#436\436]8;;\

                    INFO     ProteinFeaturiser(representation=CA,                                       ]8;id=220086;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=327957;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#438\438]8;;\
                             scalar_node_features=['amino_acid_one_hot',                                           
                             'sequence_positional_encoding', 'alpha', 'kappa'],                                    
                             vector_node_features=[], edge_types=['knn_16'],                                       
                             scalar_edge_features=['edge_distance'], vector_edge_features=[])                      

                    INFO     Instantiating task transform...                                            ]8;id=203659;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=147100;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#440\440]8;;\

                    INFO     None                                                                       ]8;id=668248;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=591229;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#444\444]8;;\

In [11]:
for name, param in model.encoder.named_parameters():
    print(name)

embedding.weight
embedding.bias
interactions.0.mlp.0.weight
interactions.0.mlp.0.bias
interactions.0.mlp.2.weight
interactions.0.mlp.2.bias
interactions.0.conv.lin1.weight
interactions.0.conv.lin2.weight
interactions.0.conv.lin2.bias
interactions.0.lin.weight
interactions.0.lin.bias
interactions.1.mlp.0.weight
interactions.1.mlp.0.bias
interactions.1.mlp.2.weight
interactions.1.mlp.2.bias
interactions.1.conv.lin1.weight
interactions.1.conv.lin2.weight
interactions.1.conv.lin2.bias
interactions.1.lin.weight
interactions.1.lin.bias
interactions.2.mlp.0.weight
interactions.2.mlp.0.bias
interactions.2.mlp.2.weight
interactions.2.mlp.2.bias
interactions.2.conv.lin1.weight
interactions.2.conv.lin2.weight
interactions.2.conv.lin2.bias
interactions.2.lin.weight
interactions.2.lin.bias
interactions.3.mlp.0.weight
interactions.3.mlp.0.bias
interactions.3.mlp.2.weight
interactions.3.mlp.2.bias
interactions.3.conv.lin1.weight
interactions.3.conv.lin2.weight
interactions.3.conv.lin2.bias
interactio

In [13]:
ckpt_path = "/home/zhang/Projects/3d/ProteinWorkshop/notebooks/outputs_schnet_pretraining(subgraph,ca_angles_pos)_best@9/checkpoints/epoch_009.ckpt"
log.info(f"Loading weights from checkpoint {ckpt_path}...")
state_dict = torch.load(ckpt_path)["state_dict"]
print(state_dict)


[07/31/24 16:12:50] INFO     Loading weights from checkpoint                                        ]8;id=110990;file:///tmp/ipykernel_72237/3296514632.py\3296514632.py]8;;\:]8;id=945351;file:///tmp/ipykernel_72237/3296514632.py#2\2]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/notebooks/outputs_schnet_pretr                
                             aining(subgraph,ca_angles_pos)_best@9/checkpoints/epoch_009.ckpt...                   

{'epoch': 9, 'global_step': 135600, 'pytorch-lightning_version': '2.3.3', 'state_dict': OrderedDict([('encoder.embedding.weight', tensor[512, 43] n=22016 (86Kb) x∈[-0.351, 0.327] μ=-0.000 σ=0.094 cuda:0), ('encoder.embedding.bias', tensor[512] 2Kb x∈[-0.180, 0.175] μ=-0.002 σ=0.079 cuda:0), ('encoder.distance_expansion.offset', tensor[50] x∈[0., 10.000] μ=5.000 σ=2.975 cuda:0), ('encoder.interactions.0.mlp.0.weight', tensor[128, 50] n=6400 (25Kb) x∈[-0.463, 0.475] μ=0.001 σ=0.126 cuda:0), ('encoder.interactions.0.mlp.0.bias', tensor[128] x∈[-0.042, 0.044] μ=-0.001 σ=0.019 cuda:0), ('encoder.interactions.0.mlp.2.weight', tensor[128, 128] n=16384 (64Kb) x∈[-0.258, 0.298] μ=0.000 σ=0.093 cuda:0), ('encoder.interactions.0.mlp.2.bias', tensor[128] x∈[-0.031, 0.035] μ=0.001 σ=0.013 cuda:0), ('encoder.interactions.0.conv.lin1.weight', tensor[128, 512] n=65536 (0.2Mb) x∈[-0.228, 0.257] μ=0.000 σ=0.064 cuda:0), ('encoder.interactions.0.conv.lin2.weight', tensor[512, 128] n=65536 (0.2Mb) x∈[-0.2

In [ ]:
# encoder
encoder_weights = collections.OrderedDict()
for k, v in state_dict.items():
    if k.startswith("encoder"):
        encoder_weights[k.replace("encoder.", "")] = v
log.info(f"Loading encoder weights: {encoder_weights}")
err = model.encoder.load_state_dict(encoder_weights, strict=True)
# model.encoder.lin_out = torch.nn.Linear(128, 1195) # here
log.warning(f"Error loading encoder weights: {err}")

In [ ]:
# from proteinworkshop.finetune import train_model

# train_model(cfg)

### 4. Wrapping up

Have any additional questions about using the components provided in the `ProteinWorkshop`? [Create a new issue](https://github.com/a-r-j/ProteinWorkshop/issues/new/choose) on our [GitHub repository](https://github.com/a-r-j/ProteinWorkshop). We would be happy to work with you to leverage the full power of the repository!

/home/yang/anaconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py